In [1]:
import pandas as pd

In [2]:
import re
import os

In [3]:
import codecs

In [4]:
from pydub import AudioSegment

In [5]:
vinbrain_folder = os.path.join('data', 'vinbrain')

In [6]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk(vinbrain_folder):
    f.extend(filenames)
    break

In [7]:
audio = []
gcp = []
vais = []
gold = []
name = []
vais_md = []

for i in f:
    ext = i.split('.')[-1]
    f_name = i.split('.')[0]
    if ext == 'wav':
        audio.append(i)
        name.append(f_name)

In [8]:
r = []
for k, i in enumerate(name):
    ff_loc = os.path.join(vinbrain_folder, i + '_google.txt')
    if os.path.isfile(ff_loc):
        gc_txt = codecs.open(ff_loc, 'r', 'UTF-8').read()
    else:
        gc_txt = None
    
    ff_loc = os.path.join(vinbrain_folder, i + '_vais.txt')
    if os.path.isfile(ff_loc):
        va_txt = codecs.open(ff_loc, 'r', 'UTF-8').read()
    else:
        va_txt = None
    
    ff_loc = os.path.join(vinbrain_folder, i + '_gold.txt')
    if os.path.isfile(ff_loc):
        go_txt = codecs.open(ff_loc, 'r', 'UTF-8').read()
    else:
        go_txt = None
    
    
    ff_loc = os.path.join(vinbrain_folder, i + '_vais_md.txt')
    if os.path.isfile(ff_loc):
        vais_md = codecs.open(ff_loc, 'r', 'UTF-8').read()
    else:
        vais_md = None
    
    f_audio = AudioSegment.from_file(os.path.join(vinbrain_folder, audio[k]))
        
    r.append((i, f_audio.duration_seconds, go_txt, va_txt, gc_txt, vais_md))

In [9]:
result = pd.DataFrame(r, columns=['id', 'duration', 'gold',  'vais', 'google', 'vais_md'])

In [10]:
from src.refine_text import normalize_golden_text, normalize_pred_text

In [11]:
result['gold_refined'] = result['gold'].apply(normalize_golden_text)

In [12]:
re.sub(r'\s+0+(.+)', r' \1', 'mỗi người có 02 chân')

'mỗi người có 2 chân'

In [13]:
re.sub(r'\s+0+(.+)', r' \1', 'vào 15/10/2020')

'vào 15/10/2020'

In [14]:
result['vais_md_refined'] = result['vais_md'].apply(normalize_pred_text)

In [15]:
result['google_refined'] = result['google'].apply(normalize_pred_text)

In [16]:
result.to_excel('/Users/bryan/Desktop/data_export.xls')

In [17]:
new_result = result[['gold_refined', 'vais_md_refined', 'google_refined']]
new_result.to_excel('/Users/bryan/Desktop/data_refined_export.xls')